# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this second notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Preparing Data

All of the data preparation will be (almost) the same as last time, so we'll very briefly detail what each code block does. See the previous notebook for a recap.

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [155]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [156]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [157]:
%%bash
python -m spacy download en
# python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


## Data Download and Preparation

### There are three QASC files:

**Dataset:**

It has following fields
- ID
- Question Stem	
- Choices
- Complete Question	
- Answer Key

**Lets understand dataset more and see how we can define dataset for our model which needs questions and answer.**
- Our model should look into the Question stem and generate answer. So if our question stem is "The sun is responsible for" then answer will be "plants sprouting, blooming and wilting"


#### QASC record

In [158]:
from requests import get, models
import json
from IPython.display import HTML

render_template = """
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
"""
models.Response._repr_html_ = lambda rsp: render_template % rsp.text

def render(jsondict):
  return HTML(render_template % json.dumps(jsondict))

In [159]:
jsondict = {"id":"3B1NLC6UGZVERVLZFT7OUYQLD1SGPZ","question":{"stem":"By what time had mouse-sized viviparous mammals evolved?","choices":[{"text":"Corvidae","label":"A"},{"text":"arthropods","label":"B"},{"text":"birds","label":"C"},{"text":"backbones","label":"D"},{"text":"keratin","label":"E"},{"text":"Jurassic","label":"F"},{"text":"front paws","label":"G"},{"text":"Parakeets.","label":"H"}]},"formatted_question":"By what time had mouse-sized viviparous mammals evolved? (A) Corvidae (B) arthropods (C) birds (D) backbones (E) keratin (F) Jurassic (G) front paws (H) Parakeets."}
render(jsondict)

In [160]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import string
import json # to read json

In [161]:
!wget http://data.allenai.org/downloads/qasc/qasc_dataset.tar.gz
!tar xvfz qasc_dataset.tar.gz
!rm qasc_dataset.tar.gz

--2020-12-29 18:24:40--  http://data.allenai.org/downloads/qasc/qasc_dataset.tar.gz
Resolving data.allenai.org (data.allenai.org)... 52.218.228.114
Connecting to data.allenai.org (data.allenai.org)|52.218.228.114|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1616514 (1.5M) [application/x-gzip]
Saving to: ‘qasc_dataset.tar.gz’

qasc_dataset.tar.gz 100%[===================>]   1.54M  2.76MB/s    in 0.6s    

2020-12-29 18:24:40 (2.76 MB/s) - ‘qasc_dataset.tar.gz’ saved [1616514/1616514]

QASC_Dataset/
QASC_Dataset/train.jsonl
QASC_Dataset/test.jsonl
QASC_Dataset/dev.jsonl


In [162]:
column_names = ["question", "answer"]
QASC_Dataset_train = pd.DataFrame(columns = column_names)
QASC_Dataset_dev = pd.DataFrame(columns = column_names)
QASC_Dataset_test = pd.DataFrame(columns = column_names)

In [163]:
## Lets read train file first/.
def _read_train(file_path):
  answer = ""
  with open(file_path, "r") as f:
    lst = []
    for line_num, line in enumerate(f):
      line = line.strip("\n")
      line = json.loads(line)
      if not line:
        continue
      question = line['question']['stem']
      answerkey = line['answerKey']
      if answerkey == 'A':
        answer = line['question']['choices'][0]['text']
      elif answerkey == 'B':
        answer = line['question']['choices'][1]['text']
      elif answerkey == 'C':
        answer = line['question']['choices'][2]['text']
      elif answerkey == 'D':
        answer = line['question']['choices'][3]['text']
      elif answerkey == 'E':
        answer = line['question']['choices'][4]['text']
      elif answerkey == 'F':
        answer = line['question']['choices'][5]['text']
      elif answerkey == 'G':
        answer = line['question']['choices'][6]['text']
      elif answerkey == 'H':
        answer = line['question']['choices'][7]['text']
      else:
        answer ="Not Available"
      lst.append([question,answer])
      # print(lst)
  df = pd.DataFrame(lst,columns = column_names) 
  # df = df.append(a_series, ignore_index=True)
  return df

In [164]:
## Lets read val file first/.
def _read_val(file_path):
  answer = ""
  with open(file_path, "r") as f:
    lst = []
    for line_num, line in enumerate(f):
      line = line.strip("\n")
      line = json.loads(line)
      if not line:
        continue
      question = line['question']['stem']
      answerkey = line['answerKey']
      if answerkey == 'A':
        answer = line['question']['choices'][0]['text']
      elif answerkey == 'B':
        answer = line['question']['choices'][1]['text']
      elif answerkey == 'C':
        answer = line['question']['choices'][2]['text']
      elif answerkey == 'D':
        answer = line['question']['choices'][3]['text']
      elif answerkey == 'E':
        answer = line['question']['choices'][4]['text']
      elif answerkey == 'F':
        answer = line['question']['choices'][5]['text']
      elif answerkey == 'G':
        answer = line['question']['choices'][6]['text']
      elif answerkey == 'H':
        answer = line['question']['choices'][7]['text']
      else:
        answer ="Not Available"
      lst.append([question,answer])
      # print(lst)
  df = pd.DataFrame(lst,columns = column_names) 
  # df = df.append(a_series, ignore_index=True)
  return df

In [165]:
## Lets read val file first/.
def _read_test(file_path):
  answer = ""
  with open(file_path, "r") as f:
    lst = []
    for line_num, line in enumerate(f):
      line = line.strip("\n")
      line = json.loads(line)
      if not line:
        continue
      question = line['question']['stem']
      lst.append([question,answer])
      # print(lst)
  df = pd.DataFrame(lst,columns = column_names) 
  # df = df.append(a_series, ignore_index=True)
  return df

In [166]:
QASC_Dataset_train = _read_train("/content/QASC_Dataset/train.jsonl")
QASC_Dataset_dev = _read_val("/content/QASC_Dataset/dev.jsonl")
QASC_Dataset_test = _read_test("/content/QASC_Dataset/test.jsonl")

In [167]:
print(f"Train Data:\n {QASC_Dataset_train.head(2)}")
print(f"Val Data:\n{QASC_Dataset_dev.head(2)}")
print(f"Test Data:\n{QASC_Dataset_test.head(2)}")

Train Data:
                                             question                       answer
0  What type of water formation is formed by clouds?                        beads
1                 Where do beads of water come from?  Vapor turning into a liquid
Val Data:
                                           question                    answer
0  Climate is generally described in terms of what?  local weather conditions
1                          what varies by altitude?  temperature and moisture
Test Data:
                                            question answer
0        What type of birth do therian mammals have?       
1  By what time had mouse-sized viviparous mammal...       


In [168]:
QASC_Dataset_train.columns

Index(['question', 'answer'], dtype='object')

In [169]:
print(f"QASC_Dataset_train Dataset size:{QASC_Dataset_train.shape}")
print(f"QASC_Dataset_dev Dataset size:{QASC_Dataset_dev.shape}")
print(f"QASC_Dataset_test Dataset size:{QASC_Dataset_test.shape}")

QASC_Dataset_train Dataset size:(8134, 2)
QASC_Dataset_dev Dataset size:(926, 2)
QASC_Dataset_test Dataset size:(920, 2)


### Data Preparation

In [170]:
import random
import torch, torchtext
from torchtext import data 

### Utility Functions

**Random Deletion**

In [171]:
# randomly delete words from a sentence with a given probability

def random_deletion(sentence, p=0.5): 
    # return if single word
    if len(sentence) == 1: 
        return sentence
    # delete words
    remaining = list(filter(lambda x: random.uniform(0,1) > p, sentence)) 
    # if nothing left, sample a random word
    if len(remaining) == 0: 
        return [random.choice(sentence)] 
    else:
        return remaining

**Random Swap**

In [172]:
# randomly swap a pair of words in a sentence for a given # of times

def random_swap(sentence, n=5): 
    if len(sentence) < 2:
      return sentence
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

**Back Translation**

In [173]:
# translate a sentence to a random language,
# and translate back to original language

def back_translate(sentence, p=0.1):
  # do nothing with probability of 1-p
  if random.uniform(0,1) > p:
    return sentence

  # combine tokenized sentence into one string
  sentence = ' '.join(sentence)

  # instantiate translator
  translator = google_translator()

  # choose a target language
  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")

  # translate to the target language
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  #print(translations)

  # translate back to original language
  translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
  #print(translations_en_random)

  # select only one translation
  if len(translations_en_random) > 1:
    translations_en_random = translations_en_random[0]

  return word_tokenize(translations_en_random)

**Carry Out Data Augmentation**

In [174]:
# for example in train.examples: 
#   example.tweets = back_translate(example.tweets, p=0.01)
#   example.tweets = random_deletion(example.tweets, p=0.1)
#   example.tweets = random_swap(example.tweets, n=1)

### Tabular File Reading

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
# from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import random
import math
import time
import torch
from torchtext import data
from torchtext import datasets
import pandas as pd

In [176]:
# Creating my home directory for keeping all the CSV files
directory = 'data'
if not os.path.exists(directory):
    os.makedirs('data')
    print("Directory '% s' created" % directory)

In [177]:
# saving all the dataframe into CSV file.
QASC_Dataset_train.to_csv("/content/data/QASC_Dataset_train.csv", index=False)
QASC_Dataset_dev.to_csv("/content/data/QASC_Dataset_dev.csv", index=False)
QASC_Dataset_test.to_csv("/content/data/QASC_Dataset_test.csv", index=False)

In [178]:
## Now lets check how many null we have in training data
QASC_Dataset_train = pd.read_csv("/content/data/QASC_Dataset_train.csv")
hotspot_trn_train_null = QASC_Dataset_train.columns[QASC_Dataset_train.isnull().any()]
QASC_Dataset_train[hotspot_trn_train_null].isnull().sum()

Series([], dtype: float64)

In [179]:
## Check for null in valid data
## Now lets check how many null we have in training data
QASC_Dataset_train = pd.read_csv("/content/data/QASC_Dataset_dev.csv")
hotspot_trn_train_null = QASC_Dataset_train.columns[QASC_Dataset_train.isnull().any()]
QASC_Dataset_train[hotspot_trn_train_null].isnull().sum()

Series([], dtype: float64)

In [180]:
from torchtext.data import Field, BucketIterator
from torchtext import data
from torchtext import datasets

In [181]:
# !shuf -n 63543 /content/data/Tweet_valid_nl.csv > /content/data/Tweet_Train_reduced.csv
# !{ echo 'text_x, text_y'; cat /content/data/Tweet_Train_reduced.csv; } > /content/data/Tweet_Train_reduced_fn.csv

In [182]:
# create Field objects
QUESTION = data.Field(
    # sequential = True,
    tokenize = 'spacy', 
    # batch_first =True,
    #  use_vocab=True,
    lower = True,
    init_token = '<sos>', 
    eos_token = '<eos>', 
    #  include_lengths=True
    )

RESPONSE = data.Field(tokenize ='spacy',
                      # sequential = True, 
                      # include_lengths=True,
                      lower = True,
                      # is_target=True,
                      init_token = '<sos>', 
                      eos_token = '<eos>', 
                      # batch_first =True
                      )

fields = [('question', QUESTION), ('answer', RESPONSE)]

In [183]:
# load the dataset in json format
train_ds, valid_ds, test_ds = data.TabularDataset.splits(
   path = 'data',
   train = 'QASC_Dataset_train.csv',
   validation = 'QASC_Dataset_dev.csv',
   test = 'QASC_Dataset_test.csv',
   format = 'csv',
   fields = fields,
   skip_header = True
)

# check an example
print(vars(train_ds[0]))

{'question': ['what', 'type', 'of', 'water', 'formation', 'is', 'formed', 'by', 'clouds', '?'], 'answer': ['beads']}


In [184]:
train_ds[0].__dict__.keys()

dict_keys(['question', 'answer'])

In [185]:
## we get an Example object. The Example object bundles the attributes of a single data point together. 
# We also see that the text has already been tokenized for us, but has not yet been converted to integers
vars(train_ds.examples[10])

{'answer': ['dehydration', 'and', 'salting'],
 'question': ['what', 'are', 'two', 'ways', 'you', 'can', 'save', 'food', '?']}

In [186]:
try:
    print(len(train_ds[10]))
except:
    print(train_ds[10].question)

['what', 'are', 'two', 'ways', 'you', 'can', 'save', 'food', '?']


In [187]:
# Before creating iterators of the Datasets we need to build the vocabulary for each Field object:
# This makes torchtext go through all the elements in the training set, check the contents corresponding to the TEXT field, 
# and register the words in its vocabulary. Torchtext has its own class called Vocab for handling the vocabulary. 
# The Vocab class holds a mapping from word to id in its stoi attribute and a reverse mapping in its itos attribute
QUESTION.build_vocab(train_ds,
                  # max_size=20000,
                  min_freq=2)
RESPONSE.build_vocab(train_ds,
                    #  max_size=20000,
                     min_freq=2
                     )

In [188]:
print(f"Unique tokens in source (QASC) vocabulary: {len(QUESTION.vocab)}")
print(f"Unique tokens in target (QASC) vocabulary: {len(RESPONSE.vocab)}")

Unique tokens in source (QASC) vocabulary: 2498
Unique tokens in target (QASC) vocabulary: 1691


In [189]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_ds, valid_ds,test_ds),
    # sort_key=lambda x: len(x.text_x), # the BucketIterator needs to be told what function it should use to group the data.
    # sort_within_batch=True,
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.question),
    sort = True,
    # repeat=False, # we pass repeat=False because we want to wrap this Iterator layer
    device = device)
# https://torchtext.readthedocs.io/en/latest/examples.html

In [190]:
b = next(iter(train_iterator)); vars(b).keys()

dict_keys(['batch_size', 'dataset', 'fields', 'input_fields', 'target_fields', 'question', 'answer'])

In [191]:
train_ds.fields

{'answer': <torchtext.data.field.Field at 0x7f84b4c67b70>,
 'question': <torchtext.data.field.Field at 0x7f84b5158588>}

In [192]:
type(train_iterator)
type(train_ds)

torchtext.data.dataset.TabularDataset

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [193]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [194]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [195]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [196]:
INPUT_DIM = len(QUESTION.vocab)
OUTPUT_DIM = len(RESPONSE.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [197]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2498, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1691, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=1691, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [198]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,390,427 trainable parameters


We initiaize our optimizer.

In [199]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [200]:
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
TRG_PAD_IDX = RESPONSE.vocab.stoi[QUESTION.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [201]:
def train(model, iterator, optimizer, criterion, clip, epoch):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # src = batch.src
        src = batch.question
        # trg = batch.trg
        trg = batch.answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        if(not i % 50):
          print(f"Epoch:{epoch}, Batch:{i}, Batch Loss:{loss.item()}")        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [202]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            src = batch.question
            # trg = batch.trg
            trg = batch.answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [203]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [204]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP, epoch)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch:0, Batch:0, Batch Loss:7.4310221672058105
Epoch:0, Batch:50, Batch Loss:4.299541473388672
Epoch:0, Batch:100, Batch Loss:4.082065105438232
Epoch: 01 | Time: 0m 2s
	Train Loss: 4.811 | Train PPL: 122.842
	 Val. Loss: 3.560 |  Val. PPL:  35.156
Epoch:1, Batch:0, Batch Loss:4.715090751647949
Epoch:1, Batch:50, Batch Loss:4.021053314208984
Epoch:1, Batch:100, Batch Loss:3.852405548095703
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.253 | Train PPL:  70.328
	 Val. Loss: 3.540 |  Val. PPL:  34.471
Epoch:2, Batch:0, Batch Loss:4.657400608062744
Epoch:2, Batch:50, Batch Loss:3.7872300148010254
Epoch:2, Batch:100, Batch Loss:3.7478435039520264
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.139 | Train PPL:  62.742
	 Val. Loss: 3.585 |  Val. PPL:  36.067
Epoch:3, Batch:0, Batch Loss:4.516617774963379
Epoch:3, Batch:50, Batch Loss:3.7915287017822266
Epoch:3, Batch:100, Batch Loss:3.856212854385376
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.037 | Train PPL:  56.674
	 Val. Loss: 3.587 |  Val. PPL:  36.113
Epoc

Finally, we test the model on the test set using these "best" parameters.

In [206]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.836 | Test PPL: 342.341 |
